# Tutorial 2: Triangular systems and Newton interpolation

In [ ]:
# Load packages:

# this package allows to work efficiently with arrays
import numpy as np
# this package is used to draw graphs
import matplotlib.pyplot as plt

We study here numerical methods for the computation of monovariate interpolation polynomials based on an arbitrarily high number of points.  

---

## Construction of the polynomials

We consider an interpolation polynomial $P_N$ of degree $N$ mapping the points $\mathbf{x} = (x_0, ..., x_N)^T$ onto $\mathbf{y} = (y_0, ..., y_N)^T$, i.e. such that 

$$P_N(x_i)=y_i \quad\text{for all} \quad i=0,\dots,N.$$

For a given basis of polynomials $\mathbf{b}(x) = (b_0(x),..., b_N(x))^T$, we are looking for the coefficients $\mathbf{c} = (c_0, ..., c_N)^T$ such that 

$$P_N = \sum\limits_{i=0}^N c_i b_i.$$

The vector $\mathbf{c}$ satisfies a linear problem of the form $A \mathbf{c} = \mathbf{y}.$ 

1) Write down the entries of the matrix $A$ in terms of the basis $\mathbf{b}$ and of the positions $\mathbf{x}$.

**Answer:** 

$$ A = \left( \mathbf{b}(x_0),\mathbf{b}(x_1), \dots, \mathbf{b}(x_N)\right)^T$$ such that $\mathbf{b}(x_i)^T \mathbf{c} = y_i$ for all $i$.

2) **Monomial decomposition**: Write down the entries of the matrix $A$ when the basis $\mathbf{b} = \mathbf{m}$ is the canonical basis $\mathbf{m}(x) = (1, x, ..., x^N)^T$. Is the computation of $\mathbf{c}$ straightforward?

**Answer:**

$$A = \left(\begin{array}{cccc} 1 & x_0 & \dots & x_0^N \\ 1 & x_1 & \dots & x_1^N  \\ \vdots & & & \vdots \\ 1 & x_N & \dots & x_N^N \end{array}\right)$$

No, we need to invert a Van der Monde matrix.

3) **Lagrange decomposition**: Recall how the Lagrange polynomials $\mathbf{l} = (l_0, ..., l_N)^T$ associated with the points $\mathbf{x}$ are defined (e.g. from MAA106). Write down the entries of the matrix $A$ when $\mathbf{b} = \mathbf{l}$.

**Answer:**

$$l_j(x) = \prod_{1 \leq i \leq N, i \neq j} \frac{x-x_i}{x_j-x_i} $$ 

We have: $A c = b$ with $A_{ij} = l_j(x_i)$, i.e. $A=I_N$, and $c=b$

4) Try to add one new point $(x_{N+1},y_{N+1})$. Can you easily express the new interpolation polynomial $P_{N+1}$ based on the previous one $P_N$?

**Answer:**

With a new point, $\forall j \neq N+1$,
$$l^{(N+1)}_j(x) := \prod_{1 \leq i \leq N+1, i \neq j} \frac{x-x_i}{x_j-x_i} = l^{(N)}_j(x).  \frac{x-x_{N+1}}{x_j-x_{N+1}}$$
And
$$l^{(N+1)}_{N+1}(x) := \prod_{1 \leq i \leq N} \frac{x-x_i}{x_{N+1}-x_i}$$
There are no further simplifications that would allow us to express easily $P_{N+1}$ as function of $P_N$

5) **Newton decomposition:** Newton suggested to work with a basis of polynomials $\mathbf{N} = (N_0, N_1, \dots, N_N)^T$ of the form 

$$ N_0(x) = 1, \qquad  N_i(x) := \prod_{j=0}^{i-1} (x-x_j). $$

Write down the entries of the matrix $A$ when $\mathbf{b} = \mathbf{N}$. 

**Answer:**

$$A = \left( \begin{array}{cccc} 1 & 0 & \dots & & 0 \\ 1 & x_1-x_0 & 0 & \dots & 0 \\ \vdots &  & & & \vdots \\  1 & x_N-x_{j} & \dots & & \prod\limits_{j=0}^{N-1} (x_N-x_j) \end{array}\right)$$

6) Express $P_{N+1}$ as a function of $P_N$, $x_0,...,x_{N+1}$ and $y_{N+1}$? 

**Answer:**
 
The polynomial $P_N \in \mathbb{R}_{N}[X]$ already satisfies $P_N(x_i) = y_i$ for $i=0,\dots,N$. But a priori $P_N(x_{N+1}) \neq y_{N+1}$. 

In order to find a polynomial of degree $N+1$ satisfying $P_N(x_i) = y_i$ for $i=0,\dots,N+1$, we can simply add to $P_N$ a polynomial that annihilates in $x_i$ for $i=0,\dots,N$ and that corrects its value in $x_{N+1}$. One of these candidates is simply $N_{N+1}$, and in order to correct the value in $x_{N+1}$, we can write 

$$ P_{N+1}(x_{N+1}) = P_N(x_N) + c_{N+1} N_{N+1}(x_{N+1})$$

which leads to (the "divided difference" formula)

$$c_{N+1} = \frac{P_{N+1}(x_{N+1}) - P_N(x_{N+1})}{N_{N+1}(x_{N+1})} = \frac{y_{N+1} - P_N(x_{N+1})}{N_{N+1}(x_{N+1})}.$$

7)  What property does $A$ satisfy ?

**Answer:**

It is triangular!

## Solving linear problem

1) a) In the following, we test our algorithm on the matrices 

$$ L = \left( \begin{array}{ccc} 1 & 0 & 0 \\ 2 & 2 & 0 \\ 3 & 5 & 1 \end{array} \right),\qquad U = \left( \begin{array}{ccc} 5 & 1 & 3 \\ 0 & 2 & 1 \\ 0 & 0 & 1 \end{array} \right).$$

Compute $L^{-1}$, $U^{-1}$ and $L^{-1}b^1$ and $U^{-1} b^2$ for $b^1 = (1,2,4)^T$ and $b^2 = (3,2,1)^T$. 

b) Implement the forward substitution algorithm to solve the problem $Lv=b$ with a lower triangular matrix $L$.

c) Test your algorithm with the matrix $L$ and the vector $b^1$ and compare your solution with the one obtained in 1.a.

d) Implement also the back substitution algorithm for $Uv=b$ with an upper triangular matrix $U$.

e) Test your algorithm with the matrix $U$ and the vector $b^2$ and compare your solution with the one obtained in 1.a.

In [ ]:
def forward_substitution(L,b):
    """
    Compute the solution of a lower triangular system
    ----------   
    parameters:
    L : lower triangular matrix (numpy array of size N,N)
    b : matrix (numpy array of size N)
    
    returns:
    V : solution of the linear problem (numpy array of size N)
    """
    N = len(b)
    V = np.zeros(N)
    for i in range(N):
        if(L[i,i] != 0):
            V[i] = (b[i] - sum(L[i,j]*V[j] for j in range(i))) / L[i,i]
        else:
            return "L is not invertible"
    return V

In [ ]:
#Test the function forward substitution with the following parameters
L    = np.array([[1,0,0],[2,2,0],[3,5,1]])
b1   = np.array([1,2,4])

sol1 = forward_substitution(L,b1)

print("solution V = ", sol1)
print("L V = ", np.matmul(L,sol1), "  b = ", b1)

In [ ]:
def back_substitution(U,b):
    """
    Compute the solution of an upper triangular system
    ----------   
    parameters:
    U : upper triangular matrix (numpy array of size N,N)
    b : matrix (numpy array of size N)
    
    returns:
    V : solution of the linear problem (numpy array of size N)
    """
    N = len(b)
    V = np.zeros(N)
    for i in range(N-1,-1,-1):
        if(U[i,i] != 0):
            V[i] = (b[i] - sum(U[i,j]*V[j] for j in range(N-1,i,-1))) / U[i,i]
        else:
            return "U is not invertible"
    return V

In [ ]:
#Test the function forward substitution with the following parameters
U    = np.array([[5,1,3],[0,2,1],[0,0,1]])
b2   = np.array([3,2,1])

sol2 = back_substitution(U,b2)

print("solution V = ", sol2)
print("U V = ", np.matmul(U,sol2), "  b = ", b2)

2) In this question, we use this algorithm to compute interpolation polynomials based on an arbitrary number of points. 

a) For the test below, we use for parameters $\mathbf{x} = (0, 1, 2)^T$ and $\mathbf{y} = (1, 3, -1)^T$.
- With this choice of $\mathbf{x}$, what are the Newton polynomials $\mathbf{N}$ (as defined in I.5)?
- Compute (on paper) the coefficients $\mathbf{c}$ in that basis, i.e. such that $P_2 = c_0 N_0 + c_1 N_1 + c_2 N_2$ is the polynomial of degree 2 satisfying $P_2(x_i) = y_i$ for $i=0,1,2$. 

b) Construct a function that computes the coefficients $\mathbf{c}$ of the polynomials $P_N$ in the Newton basis $\mathbf{b} = \mathbf{N}$. 

***Indication:*** you can use one of the functions of the last question for solving a triangular system. 

c) Test your function with the parameters in a) and compare your numerical result with the one expected. 

d) Construct a function that returns $P_N(x)$ based on the abscissa $x$, the coefficients $\mathbf{c}$ of the decomposition $P_N$ in the basis $\mathbf{N}$ and the locations $\mathbf{x} = (x_0,\dots,x_N)^T$.

e) Plot $P_N(x)$ as a function of $x$ using the parameters given in a) and with the coefficients $\mathbf{c}$ obtained in c). Does it pass in $y_i$ at $x_i$ at all $i=0,1,2$?

**Answer:**

a) It is 1, 2 and -3. Indeed, $P_2(x_0) = y_0 = 1 = c_0$, then $P_2(x_1) = y_1 = 3 = c_0 \times 1 + c_1 \times (x_1-x_0) = 1 + c_1$, then $c_1 = 2$. Finally, $P_2(x_2) = -1 = 1 + c_1 (x_2-x_0) + c_2 (x_2-x_0)(x_2-x_1) = 1+4+2c_2$ which provides $c_2 = \frac{-6}{2} = -3$.

In [ ]:
#b)
def compute_coeff_Newton(vect_x, vect_y):
    """
    Compute the coefficients of the interpolation polynomial that takes value vect_y in vect_x in Newton's basis
    ----------   
    parameters:
    vect_x : vector of abscissa (x_0, ..., x_N)
    vect_y : vector of values (y_0, ..., y_N) = (P_N(x_0), ..., P_N(x_N))
    
    returns:
    coeff : vector of coefficients of the interpolation polynomial in Newton's basis 
    """
    
    N = len(vect_x)
    
    # compute the lower triangular matrix L
    L = np.zeros((N, N))
    L[:, 0] = 1
    for i in range(1, N):
        for j in range(1, i+1):
            L[i, j] = L[i, j-1] * (vect_x[i]-vect_x[j-1])
            
    # solve the problem
    coeff= forward_substitution(L,vect_y)
     
    return coeff

In [ ]:
#c)
# Test your algorithm with the following parameters
vect_x = np.array([0, 1, 2])
vect_y = np.array([1, 3, -1])

coeff = compute_coeff_Newton(vect_x, vect_y)
print("coefficients in Newton's basis",coeff)

In [ ]:
#d)
def interpolation_polynomial(x, coeff, vect_x):
    """
    Compute the coefficients of the interpolation polynomial that takes value vect_y in vect_x in Newton's basis
    ----------   
    parameters:
    x      : vector of abscissa where the function is evaluated in the plot
    coeff  : vector of coefficients of the interpolation polynomial in Newton's basis 
    vect_x : vector of interpolation abscissa (x_0, ..., x_N)
    """
    val = np.array([])
    for y in x:
        z = coeff[0]
        for i in range(1, len(vect_x)):
            update = 1
            for j in range(i):
                update *= (y - vect_x[j])
            z += coeff[i]*update
        val = np.append(val,z)
    return val

In [ ]:
#e)
# Test your algorithm with the following parameters
vect_x = np.array([0, 1, 2])
vect_y = np.array([1, 3, -1])

coeff_interp  = compute_coeff_Newton(vect_x, vect_y)


# vector of abscissa where the function is evaluated in the plot
x_plot = np.linspace(0, 2, 200)
# vector of evaluation P_N(x) in the plot
y_plot = interpolation_polynomial(x_plot, coeff_interp, vect_x)

# plot the interpolation polynomial as a function of x
plt.figure(1)
plt.plot(   x_plot, y_plot, '--', color='blue', label="Your interpolation")
plt.scatter(vect_x, vect_y,       color='red',  label="Interpolation points")
plt.legend()
plt.show()

**To go further**

3) Compare the number of operations required to compute this interpolation polynomials in Lagrange and in Newton's basis. What are the advantages and drawbacks of both methods?

**Answer:**

4) Can you modify the basis polynomials $N_i$ in order to keep the determinant of $A$ independant of $N$.
Can you propose other basis to reduce the number of operations? 

**Answer:**

5) For triangular matrices, the condition number is defined as $Cond(A) = \left(\max\limits_i |A_{i,i}|\right) \left(\max\limits_i \frac{1}{|A_{i,i}|}\right)$. Compute the condition number for the Newton interpolation matrix. Play with diagonal coefficients $A_{i,i}$ and write an interpretation of the influence of the condition number on the result of the present algorithm. What is the other advantage of the choice of question 4?